[referensi 1](https://github.com/jkhlr/hybrid-recommender-system) <br>
[referensi 2](https://en.wikipedia.org/wiki/Recommender_system#Hybrid_recommender_systems)

After searching some hybrid(s) recommendation systems problem, i cannot find some good example to read. 
So i used `referensi1` to become my use case and i added some analysis to my github.

---

# Hybrid Recommender Systems
Most recommender systems now use a hybrid approach, combining collaborative filtering, content-based filtering, and other approaches. There is no reason why several different techniques of the same type could not be hybridized.<br>
Hybrid approaches can be implemented in several ways: by making content-based and collaborative-based predictions separately and then combining them; by adding content-based capabilities to a collaborative-based approach (and vice versa); or by unifying the approaches into one model (see for a complete review of recommender systems).<br>
Several studies that empirically compare the performance of the hybrid with the pure collaborative and content-based methods and demonstrated that the hybrid methods can provide more accurate recommendations than pure approaches. These methods can also be used to overcome some of the common problems in recommender systems such as cold start and the sparsity problem, as well as the knowledge engineering bottleneck in knowledge-based approaches. <br> 
Netflix is a good example of the use of hybrid recommender systems.The website makes recommendations by comparing the watching and searching habits of similar users (i.e., collaborative filtering) as well as by offering movies that share characteristics with films that a user has rated highly (content-based filtering). <br>

In [0]:
#!pip install surprise

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Library
Library that what i used. 'Surprise' is a Python scikit building and analyzing recommender systems that deal with explicit rating data. 

In [0]:
import numpy as np
import pandas as pd 
import surprise as sp

In [53]:
parsed_data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [68]:
wines_all.head()

,id,country,province,region,variety,price
0,1,Argentina,Mendoza Province,Agrelo,Bonarda,17.857143
1,2,Argentina,Mendoza Province,Agrelo,Bordeaux-style Red Blend,34.500000
2,3,Argentina,Mendoza Province,Agrelo,Cabernet Franc,40.800000
3,4,Argentina,Mendoza Province,Agrelo,Cabernet Sauvignon,20.250000
4,5,Argentina,Mendoza Province,Agrelo,Chardonnay,15.000000


In [0]:
# data parsing
parsed_data = pd.read_csv("/content/drive/My Drive/data/winemag-data-130k-v2.csv")
filtered_data = parsed_data[['country','province','region_1','variety','price','taster_name','points']]
cleaned_data = filtered_data.rename(columns={'region_1': 'region'}).dropna(subset=['country','province','region','variety','taster_name','points'])

# group all wines from a region that have the same variety, assign mean price
wines_all = cleaned_data.groupby(['country', 'province', 'region', 'variety']).agg({'price': 'mean'}).reset_index()
wines_all = wines_all.assign(id=pd.Series(range(1, wines_all.shape[0]+1), dtype=int, index=wines_all.index))
wines_all = wines_all[['id', 'country', 'province', 'region', 'variety', 'price']]

users_all = cleaned_data.groupby('taster_name').count().reset_index()[['taster_name']]
users_all = users_all.assign(id=pd.Series(range(1, users_all.shape[0]+1), dtype=int, index=users_all.index))

# link ratings to wines and users via id
wine_id_translator = {(row['country'], row['province'], row['region'], row['variety']): row['id'] for index, row in wines_all.iterrows()}
user_id_translator = {row['taster_name']: row['id'] for index, row in users_all.iterrows()}
def get_wine_id_series(data_frame):
    return pd.Series((wine_id_translator[(row['country'], row['province'], row['region'], row['variety'])] for _, row in data_frame.iterrows()), index=data_frame.index)
def get_user_id_series(data_frame):
    return pd.Series((user_id_translator[row['taster_name']] for _, row in data_frame.iterrows()), index=data_frame.index)

# aggregate average points of all ratings from a user for a wine
ratings_all = cleaned_data.assign(wine_id=get_wine_id_series, user_id=get_user_id_series)[['taster_name', 'user_id', 'wine_id', 'points']].groupby(['user_id', 'taster_name', 'wine_id']).mean().reset_index()

# only include wines that have 3 or more ratings
most_rated_wines = list(ratings_all.groupby(['wine_id']).count()[lambda x: x['points'] >= 3].reset_index()['wine_id'].values)

ratings = ratings_all.loc[ratings_all['wine_id'].isin(most_rated_wines)].astype({'wine_id': int, 'user_id': int}).reset_index(drop=True)
wines = wines_all.loc[wines_all['id'].isin(most_rated_wines)].astype({'id': int}).reset_index(drop=True)
users = users_all.loc[users_all['id'].isin(ratings['user_id'].values)].astype({'id': int}).reset_index(drop=True)

## Wines

In [56]:
wines.head()

,id,country,province,region,variety,price
0,739,Canada,Ontario,Niagara Peninsula,Riesling,42.423077
1,741,Canada,Ontario,Niagara Peninsula,Vidal Blanc,62.615385
2,757,France,Alsace,Alsace,Gewürztraminer,34.206897
3,760,France,Alsace,Alsace,Pinot Blanc,17.622047
4,778,France,Alsace,Crémant d'Alsace,Sparkling Blend,24.886256


## Ratings

In [57]:
ratings.head()

,user_id,taster_name,wine_id,points
0,1,Alexander Peartree,5069,87.666667
1,1,Alexander Peartree,5737,89.000000
2,1,Alexander Peartree,5738,86.750000
3,1,Alexander Peartree,5741,86.250000
4,1,Alexander Peartree,5743,88.000000


## Users

In [58]:
users.head()

,taster_name,id
0,Alexander Peartree,1
1,Anna Lee C. Iijima,2
2,Anne Krebiehl MW,3
3,Carrie Dykes,4
4,Christina Pickard,5


## Collaborative Filtering
`predict_cf` returns the predicted rating of the user with name `taster_name` for item with id `wine_id`.
The function uses a KNN classifier. To train the model, all other ratings are used.
The error and the actual rating is returned as well.

In [0]:
# Collaborative Filtering

def predict_cf(ratings, taster_name, wine_id):
    is_target = (ratings['taster_name'] == taster_name) & (ratings['wine_id'] == wine_id)
    target = ratings[is_target].iloc[0]
    
    train_set = sp.Dataset.load_from_df(
        ratings[~is_target][['user_id', 'wine_id', 'points']], 
        sp.Reader(rating_scale=(0, 100))
    ).build_full_trainset()

    algo = sp.KNNBasic(verbose=False)
    algo.fit(train_set)
    prediction = algo.predict(target['user_id'], target['wine_id'], verbose=False)
    return prediction.est, prediction.est - target['points'], target['points']

## Content-Based
`predict_cn` returns the predicted rating of the user with name `taster_name` for item with id `wine_id`.
The function also uses a KNN classifier. To train the model, all other ratings from the same user, as well as the wine database are used.
The error and the actual rating is returned as well.

In [0]:
# Content-Based

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

def predict_cn(ratings, wines, taster_name, wine_id):
    user_ratings = ratings[ratings['taster_name'] == taster_name].join(wines.set_index('id'), on='wine_id')
    is_target = (user_ratings['wine_id'] == wine_id)
    
    features = pd.get_dummies(user_ratings.drop(columns=['points']))
    train_features = features[~is_target]
    target_features = features[is_target]
    
    encoder = LabelEncoder()
    train_labels = encoder.fit_transform(user_ratings[~is_target]['points'])
    target_label = user_ratings[is_target]['points'].iloc[0]

    clf = KNeighborsClassifier(n_neighbors=1)
    clf.fit(train_features, train_labels)
    prediction = encoder.inverse_transform(clf.predict(target_features))[0]
    return prediction, prediction - target_label, target_label

## Testing the Recommenders

In [0]:
def test_classifier(taster_name, wine_id):
    pred_cf, error_cf, truth = predict_cf(ratings, taster_name, wine_id)
    pred_cn, error_cn, truth = predict_cn(ratings, wines, taster_name, wine_id)
    print("Results for {} on wine with id {}:".format(taster_name, wine_id))
    print("Collaborative Filtering: \t prediction: {:.5f} \t error: {:.5f}".format(pred_cf, error_cf))
    print("Content-Based: \t\t\t prediction: {:.5f} \t error: {:.5f}".format(pred_cn, error_cn))

In [73]:
test_classifier(taster_name='Anna Lee C. Iijima', wine_id=741)

Results for Anna Lee C. Iijima on wine with id 741:
Collaborative Filtering: 	 prediction: 89.65560 	 error: -0.01107
Content-Based: 			 prediction: 89.50000 	 error: -0.16667


Hasil dari kolaboratif filtering adalah wine dengan id 741 akan menghasilkan skor 89.65 dengan error -0.011. Ini menunjukkan bahwasanya prediksi menghasilkan nilai yang cukup baik dibandingkan dengan nilai rating wine aslinya. <br>

Selanjutnya untuk content-based, hasil prediksinya tidak jauh berbeda dengan kolaboratif filtering. Walaupun, error yang dihasilkan jauh lebih tinggi dibandingkan dengan rekomendasi dari kolaboratif.

## Weighted Hybrid Recommender
Create a weighted hybrid recommender, combining the results of `predict_cf` and `predict_cn`. The weights should be static.

In [64]:
def predict_weighted(ratings, wines, taster_name, wine_id):
    prediction_cf, _, truth = predict_cf(ratings, taster_name, wine_id)
    prediction_cn, _, truth = predict_cn(ratings, wines, taster_name, wine_id)
    
    # Weights can be chosen differently, depending on 
    # the (assumed) quality of the recommenders
    prediction = 0.5 * prediction_cf + 0.5 * prediction_cn
    error = prediction - truth
    return prediction, error, truth


pred_weighted, error_weighted, truth = predict_weighted(ratings, wines, taster_name='Anna Lee C. Iijima', wine_id=741)
print("Weighted Hybrid: \t prediction: {:.5f} \t error: {:.5f}".format(pred_weighted, error_weighted))

Weighted Hybrid: 	 prediction: 89.57780 	 error: -0.08887


Secara teori, seharusnya hybrid lebih baik dibandingkan dengan metode content-based dan kolaboratif filtering. Tetapi pada percobaan ini tidak dihasilkan hasil yang seharusnya. Hipotesis saya adalah beban yang diberikan (0.5) tidak seharusnya sama antar metode, ini yang menyebabkan error lebih tinggi dibandingkan dengan kolaboratif-filtering.